In [1]:
# prompt: add drive mount code

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Sun Dec 22 20:21:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0              46W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
%pwd

'/content'

In [4]:
%ls /content/drive/MyDrive/bootcamp/animal_classification/

config/  README.md  requirements.txt  src/


In [5]:
#drive da çalışalım : /content/drive/MyDrive/bootcamp/animal_classification/
%cd /content/drive/MyDrive/bootcamp/animal_classification/

/content/drive/MyDrive/bootcamp/animal_classification


In [ ]:
#sudo mount --bind /source_folder /destination_folder
#!sudo mount --bind /content/drive/MyDrive/bootcamp/animal_classification /content/animal_classification

In [6]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
rrebirrth_animals_with_attributes_2_path = kagglehub.dataset_download('rrebirrth/animals-with-attributes-2')

print('Data source import complete.')

100%|██████████| 13.0G/13.0G [01:02<00:00, 224MB/s]

Extracting files...


Data source import complete.


In [8]:
%pwd

'/content/drive/MyDrive/bootcamp/animal_classification'

In [9]:
import os
import shutil
from PIL import Image

# Initialize the dictionary to hold paths for each animal
image_paths = {}

# Base directory for your dataset
base_path = "/root/.cache/kagglehub/datasets/rrebirrth/animals-with-attributes-2/versions/1/Animals_with_Attributes2/JPEGImages/"

# List of animals to search for
animals = ["collie", "dolphin", "elephant", "fox", "moose", "rabbit", "sheep", "squirrel", "giant+panda", "polar+bear"]

# Traverse the directory structure to collect image paths
for dirname, _, filenames in os.walk(base_path):
    for animal in animals:
        if animal in dirname:
            if animal not in image_paths:
                image_paths[animal] = []
            for filename in filenames:
                image_paths[animal].append(os.path.join(dirname, filename))

datasets_dir = "dataset/raw"
# Create the 'dataset/raw' directory if it doesn't exist
os.makedirs(datasets_dir, exist_ok=True)

# Copy the first 650 images for each animal
for animal, paths in image_paths.items():
    # Create a directory for the current animal within 'dataset/raw'
    animal_dir = os.path.join(datasets_dir, animal)
    os.makedirs(animal_dir, exist_ok=True)

    # Copy the first 650 images
    for i, path in enumerate(paths[:650]):
        destination_path = os.path.join(animal_dir, os.path.basename(path))
        shutil.copy(path, destination_path)

print("Images copied successfully!")

# Get a list of all class names (subdirectories) within the base directory
class_names = [f for f in os.listdir(datasets_dir) if os.path.isdir(os.path.join(datasets_dir, f))]

# Loop through each class name and count the images
for class_name in class_names:
    # Construct the path to the current class's directory
    class_dir = os.path.join(datasets_dir, class_name)

    # Initialize counters for total and empty images
    total_images = 0
    empty_images = 0

    # Loop through files in the class directory
    for filename in os.listdir(class_dir):
        # Construct the full file path
        file_path = os.path.join(class_dir, filename)

        # Check if it's a file (not a subdirectory)
        if os.path.isfile(file_path):
            total_images += 1

            # Attempt to open the image using Pillow
            try:
                with Image.open(file_path) as img:
                    # Check if the image is empty (size is 0)
                    if img.size == (0, 0):
                        empty_images += 1
            except IOError:
                # Handle cases where the file is not a valid image
                print(f"Warning: Could not open or read image file: {file_path}")

    # Print the class name, total images, and empty image count
    print(f"{class_name}: {total_images} images, {empty_images} empty")

Images copied successfully!
giant+panda: 650 images, 0 empty
polar+bear: 650 images, 0 empty
rabbit: 650 images, 0 empty
elephant: 650 images, 0 empty
collie: 650 images, 0 empty
squirrel: 650 images, 0 empty
sheep: 650 images, 0 empty
fox: 650 images, 0 empty
moose: 650 images, 0 empty
dolphin: 650 images, 0 empty


In [10]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [11]:
!pip show tensorflow
!pip show tensorflow-gpu

Name: tensorflow
Version: 2.17.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tf_keras


In [ ]:
!python src/train.py

2024-12-22 20:28:15.998804: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-22 20:28:16.017778: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-22 20:28:16.023545: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-22 20:28:17.231449: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
--------------------Dataset Bilgileri--------------------
Sınıf sayısı: 10
Sınıflar: ['collie', 'dolphin', 'elephant', 'fox', 'moose', 'rabbit', 'sheep', 'squirrel', 'giant+panda', 'polar+bear']
Eksik klasör: dataset/processed/train

Veri se